# OpenAI Function Calling In LangChain


In [1]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [2]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [3]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [4]:
foo.name

'Joe'

In [5]:
foo = User(name=100,age=32, email="joe@gmail.com")

In [6]:
from pydantic import BaseModel
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [7]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

As expected, we can see that Pydantic returns a validation error when we pass a string to an int field.

In [8]:
foo_n = pUser(name="Joe", age="hello", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)

We can also nest pydantic models as type hints for fields

In [9]:
from typing import List
class Class(BaseModel):
    students: List[pUser]

In [10]:
obj = Class(students=[pUser(name="Jane", age=32, email="jane@gmail.com")])

In [11]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


Let's define a Pydantic data class for a made-up function, `WeatherSearch`.

This function is returning a set of parameters required for an API call to our made-up weather API.

In [15]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [13]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

We can easily convert this into a [dictionary](https://platform.openai.com/docs/guides/gpt/function-calling) that specifies the function.

In [14]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [17]:
# We require it have a description
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

convert_pydantic_to_openai_function(WeatherSearch1)

KeyError: 'description'

In [18]:
# We require it have a description
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch2',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code', 'type': 'string'}},
  'required': ['airport_code']}}

In [19]:
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI()
model.invoke("What is the weather in San Francisco right now?",
             functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

In [20]:
model_with_function = model.bind(functions=[weather_function])

In [21]:
model_with_function.invoke("What is the weather in San Francisco right now?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

We can look at the [LangSmith trace](https://smith.langchain.com/public/52170085-723a-4945-936c-d394d1123de0/r) to see what happened: 

* OpenAI correctly decides to use the function based on the context of our input question
* OpenAI correctly returns the agurments (`airport_code`) for our make-up weather API.

This is correctly calling the above function to produce the input to our made-up API.


## Forcing it to use a function

We can force the model to use a function

In [22]:
model_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [23]:
model_forced_function.invoke("What is the weather in San Francisco right now?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [24]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helfpul assistant"),
        ("user", "{input}")
    ]
)


In [25]:
chain = prompt | model_forced_function

In [26]:
chain.invoke({"input": "What is the weather in San Francisco right now?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [27]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [28]:
convert_pydantic_to_openai_function(ArtistSearch)

{'name': 'ArtistSearch',
 'description': 'Call this to get the names of songs by a particular artist',
 'parameters': {'title': 'ArtistSearch',
  'description': 'Call this to get the names of songs by a particular artist',
  'type': 'object',
  'properties': {'artist_name': {'title': 'Artist Name',
    'description': 'name of artist to look up',
    'type': 'string'},
   'n': {'title': 'N', 'description': 'number of results', 'type': 'integer'}},
  'required': ['artist_name', 'n']}}

In [29]:
model = ChatOpenAI()

functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

model_with_functions = model.bind(functions=functions)

In [30]:
model_with_functions.invoke("What is the weather in SF?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

In [31]:
model_with_functions.invoke("What are three songs by Taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{\n  "artist_name": "Taylor Swift",\n  "n": 3\n}'}})